# Estimate data load of dictionary expansion method

This notebook takes the Dolinsky-Huber-Horne (DHH) social group keywords dictionary and estimates how many terms would require manual coding if one applied word-embedding based dictionary expansion to discover new keywords not yet in the dictionary.

Word-embedding based dictionary expansion is a method for finding likely relavant keywords based on a set of "seed" keywords.
For each dictionary keyword, one uses the embedding model to find the $k$ most similar words (according to cosine similarity) to the keyword.
These $k$ words are candidates for addition to the dictionary. 

Importantly, while automating the search for keyword *candidate*, its a best practice to manually review candidate terms to decide whether or not a given term should be included in the dictionary.
Thus, word-embedding based dictionary expansion requires (expert) coding (a.k.a. supervision).

This need for a human in the loop implies that this method is not free from manual labor.
The amount of manual labor required to implement it, in turn, increases in (i) $n$, the number of seed keywords in the dictionary, and $k$, the number candidate terms that should be considered for each seed keyword.

In our analysis, $n$ is fixed because we start with the DHH dictionary.
Hence we vary $k$ between 10 and 100 to compute the number of words that required manual coding in the dictionary expansion process.


In [ ]:
from types import SimpleNamespace

args = SimpleNamespace()

args.experiment_name = 'dictionary_expansion'
args.experiment_results_path = '../../../results/validation/dhh_dictionary'

args.keywords_file = '../../../data/exdata/dhh_dictionary/keywords.csv'
args.gensim_embedding_model = 'word2vec-google-news-300'
args.nearest_neighbors_values = '10,25,50,100'

args.output_folder = '../../../data/validation/dhh_dictionary/dictionary_expansion'
args.overwrite_output = False

# parse the arguments

args.nearest_neighbors_values = [int(x.strip()) for x in args.nearest_neighbors_values.split(',')]
args.nearest_neighbors_values = sorted(args.nearest_neighbors_values)

In [5]:
import os
import json

import pandas as pd
import re

import gensim.downloader as api

from tqdm.auto import tqdm

from typing import List, Tuple

## Load dictionary and prepare the keywords

Keywords in the DHH dictionaries use glob patterns. 
We need to convert them to regex.

Moreover, any words in the embedding model's vocabulary that are n-grams with n ≥ 2 do not contain under scores as token delimiter (not whitespaces).
Hence, we need to pre-process the dictionary keywords accordingly to allow mapping them to the word vectors.

In [9]:
# load the dictionary
keywords_wide = pd.read_csv(args.keywords_file)

In [10]:
# convert the glob keywords into regex and make them compatible with encoding style of word2vec vocabulary (e.g., whitespaces are _ in n-grams)
def glob_to_regex(glob):
    """
    Convert a glob pattern to a regular expression.
    """
    # Escape all characters that are special in regex, except for *, ?, and []
    regex = re.escape(glob)
    # make compatible with word2vec encoding
    regex = regex.replace('\\ ', '_')
    
    # Replace the escaped glob wildcards with regex equivalents (given encoding of word2vec vocabulary)
    regex = regex.replace(r'\*', '[^_]*?')
    regex = regex.replace(r'\?', '.')
    regex = regex.replace(r'\[', '[')
    regex = regex.replace(r'\]', ']')
    
    # Add anchors to match the entire string
    regex = r'^' + regex + '$'

    
    return regex

keywords = {c: [glob_to_regex(v.strip()) for v in vals if not pd.isna(v)] for c, vals in keywords_wide.T.iterrows()}

## load a pre-trained Word2Vec embedding model

In [11]:
model = api.load(args.gensim_embedding_model)  # This will (down)load the Google News pre-trained Word2Vec model
# get the model vocabulary
model_vocab = list(model.key_to_index.keys())

## Find dictionary keywords' word vectors

To look up similar terms for a dictionar keyword in the embedding model, we need to find each dictionary keyword's word vectors.
This requires checking whether the keyword is in the embedding model's vocabulary.

One problem of applying the dictionary expansion method to dictionaries that contain multi-word expressions is that the concreate $n$-grams recorded in the dictionary are not in the embedding model's vocabulary.

Below, we address this issue by recursively chunking a keyword that cannot be found in the embedding model's dictionary into its constituent $n$-grams and checking whether they can be found in the vocabulary. We repear this recursive chunking up to three times.

**_Example 1_**:

The bi-gram keyword "business_people" is not in the vocabulary.
We chunk it into its uni-gram words (splitting at the token separator '_') and check whether each is in the vocabulary.

**_Example 2_**:

The 7-gram keyword "people_living_and_working_in_rural_areas" is not in the vocabulary.
We first chunk it into its constituent 6-grams ("people_living_and_working_in_rural" and "living_and_working_in_rural_areas") and check whether they are in the vocabulary. None is.
So we chunk each into its constituent 5-grams and check whether these are in the vocabulary.
Etc.


In [12]:
# helper functions
def find_in_vocabulary(term: str, model_vocab: List[str]) -> List[Tuple[int, str]]:
    return [(i, w) for i, w in enumerate(model_vocab) if re.search(term, w)]
# # Example
# find_in_vocabulary('^worker[^_]*?$', model_vocab)

def split(x: str, pattern):
    return re.split(pattern, x)

def segment_into_ngrams(x: str, n: int=1, pattern: str='(?<!^)_(?!\])', sep: str='_') -> List[str]:
    x = split(x, pattern)
    if len(x) < n:
        return [x]
    elif n > len(x):
        return [sep.join(x)]
    else:
        out = ['^' + sep.join(x[i:i+n]) + '$' for i in range(0, len(x)-n+1)]
        out[0] = out[0].replace('^^', '^')
        out[-1] = out[-1].replace('$$', '$')
        out = [s for s in out if s != '^[^_]*?$']
        return out
# # Examples
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=1))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=2))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=3))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=4))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=5))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=6))
# print(segment_into_ngrams("people_living_and_working_in_rural_areas", n=7))

def find_keyword_in_vocabulary(
        kw: str, 
        model_vocab: List[str],
        max_splits: int=3
    ) -> List[Tuple[str, str, int, int, str]]:
    """Find dictionary keyword in word2vec model vocabulary.

    Args:
        kw (str): dictionary keyword (regex pattern)
        model_vocab (List[str]): word2vec model vocabulary

    Returns:
        List[Tuple[str, str, int, int, str]]: list of tuples with keyword, keyword segment, times splitted, index of matched term in vocabulary, and vocabulary word
    """
    vocab_terms = find_in_vocabulary(kw, model_vocab)
    n = len(split(kw, pattern='(?<!^)_(?!\])'))
    if len(vocab_terms) > 0:
        return [(kw, None, 0, t[0], t[1]) for t in vocab_terms]
    terms = [kw]
    vocab_terms = []
    while len(terms) > 0:
        term = terms.pop(0)
        n_ = len(split(term, pattern='(?<!^)_(?!\])'))-1
        if n-n_ > max_splits:
            break
        if n_ == 0:
            continue
        segs = segment_into_ngrams(term, n=n_)
        for s in segs:
            tmp = find_in_vocabulary(s, model_vocab)
            if len(tmp) > 0:
                tmp = [(kw, s, n-n_, t[0], t[1]) for t in tmp]
                vocab_terms.extend(tmp)
            else:
                terms.append(s)
    return vocab_terms
# # examples
# print(find_keyword_in_vocabulary('^employee[^_]*?$', model_vocab))
# print(find_keyword_in_vocabulary('^employer[^_]*?$', model_vocab))
# print(find_keyword_in_vocabulary('^business_people$', model_vocab))

In [ ]:
# get matches for words in the dictionary
# note: this can take 20-25 minutes
matches = []
for cat, kws in tqdm(keywords.items(), desc='searching matches for dictionary keywords in model vocabulary'):
    for kw in kws:
        matched = find_keyword_in_vocabulary(kw, model_vocab)
        matches.extend([(cat, ) + m for m in matched])

searching matches for dictionary keywords in model vocabulary:   0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
# convert the result of this process to a data frame
cols = ['category', 'keyword', 'keyword_segment', 'keyword_splitted_n_times', 'match_vocabulary_index', 'match_vocabulary_word']
matched_df = pd.DataFrame(matches, columns=cols)

,category,keyword,keyword_segment,keyword_splitted_n_times,match_vocabulary_index,match_vocabulary_word
0,var1,^employee[^_]*?$,None,0,725,employees
1,var1,^employee[^_]*?$,None,0,2019,employee
2,var1,^employee[^_]*?$,None,0,758697,employeed
3,var1,^employee[^_]*?$,None,0,1312688,employees.The
4,var1,^employee[^_]*?$,None,0,1511655,employeers


The data frame records the following information:

- `category`: the dictionary category of the keyword
- `keyword`: the keyword (in its pre-processed form)
- `keyword_segment`: the segment of the keyword that was found in the embedding model's vocabulary. None if ``keyword`` itsself was found in the vocabulary.
- `keyword_splitted_n_times`: the number of times the $n$-gram keyword was splitted to find matches in the embedding model's vocabulary.
- `match_vocabulary_index`: the index of the keyword (segment) in the embedding model's vocabulary.
- `match_vocabulary_word`: the word matched to the keyword (segment) in the embedding model's vocabulary.


## Find nearest neighbors

For each keyword (segment) matched to a word in the word embedding model's vocabulary, we find the $k$ nearest neighbors in the embedding model's vocabulary.
Below we first find the top-100 most similar words and below check how reducing $k$ from 100 to 50, 25, and 10 affects the numbers of terms that would require manual checking.

In [20]:
k = max(args.nearest_neighbors_values)
nearest_neighbors = [
    (c, w, v, s, r) 
    for c, w in tqdm(
        zip(matched_df['category'], matched_df['match_vocabulary_word']),
        desc='searching nearest neighbors for matched model vocabulary terms',
        total=len(matched_df),
    )
    for r, (v, s) in enumerate(model.most_similar(w, topn=k))
]

searching nearest neighbors for matched model vocabulary terms:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
# convert to DataFrame
nearest_neighbors_df = pd.DataFrame(nearest_neighbors, columns=['category', 'word', 'neighbor', 'similarity', 'rank'])
nearest_neighbors_df['neighbor'] = nearest_neighbors_df.neighbor.str.lower()

## Compute number of unique words in nearest neighbors set that would require manual review

In [25]:
os.makedirs(args.output_folder, exist_ok=True)
keywords_file_template = 'dhh_dictionary_expanded_keywords_k{k}.json'

df_to_json = lambda x: {c: words[0].tolist() for c, words in x.groupby('category').agg({'neighbor': lambda x: x.str.replace('_', ' ')}).iterrows()}

counts = dict()
for k in args.nearest_neighbors_values:
    # subset to k nearest neighbors
    tmp = nearest_neighbors_df[nearest_neighbors_df['rank'] < k]
    
    # save keywords to file
    fp = os.path.join(args.output_folder, keywords_file_template.format(k=k))
    if not os.path.exists(fp) or args.overwrite_output:
        with open(fp, 'w') as f:
            json.dump(df_to_json(tmp), f, indent=2)

    # group by `neighbor` and count the number of times each neighbor appears and its mean similarity
    unique_neighbors_df = tmp.groupby('neighbor').agg({'word': 'count', 'similarity': 'mean'}).sort_values('similarity', ascending=False).reset_index().rename(columns={'word': 'n_times', 'similarity': 'mean_similarity', 'neighbor': 'word'})
    
    # flag words that already appear in the dictionary
    kws = [val for vals in keywords.values() for val in vals]
    unique_neighbors_df['in_dictionary'] = unique_neighbors_df.word.apply(lambda x: any([bool(re.search(re.compile(kw, flags=re.IGNORECASE), x)) for kw in kws]))
    
    # tabulate existing/new words
    counts[k] = unique_neighbors_df.in_dictionary.value_counts()

In [27]:
result = pd.DataFrame(counts)
result.index.names = ['in_dictionary']

The table shows that if one would find the top-10 most similar words with the embedding model for each dictionary keyword (segment), there would be 4119 unique terms that would require manual checking (242 words in the nearest neighbors set would require no checking because they already in the set of dictionary keywords).
The number of unique terms that would require manual checking increases to 10100 if one would increase $k$ to 25 to examine the 25 (not 10) most similar terms to each dictionary keyword (segment).
(This is alraedy more than the number of labeled sentences we need for our fine-tuning approach.)
At latest when one wants to examine the 50 most similar terms to each dictionary keyword (segment), the number of unique terms that would require manual checking approaches 20K.

In [ ]:
dest = os.path.join(args.experiment_results_path, args.experiment_name)
os.makedirs(dest, exist_ok=True)
fp = os.path.join(dest, 'dictionary_expansion_words_to_review_estimates.csv')
if not os.path.exists(fp) or args.overwrite_output:
    result.to_csv(fp)